# Домашнее задание к теме Базовые понятия статистики

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики.

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

7. Информация об атрибутах (ключевые параметры):

  4:  rectal temperature — Ректальная температура  (числовой)  
          - линейная  
          - в градусах Цельсия  
          - повышенная температура возможна при инфекции  
          - может снижаться при поздней стадии шока  
          - нормальная температура — 37.8  
          - этот параметр обычно меняется по мере развития проблемы  
               например: может начинаться нормальной, затем повышаться из-за  
                   поражения, снова становиться нормальной, когда лошадь впадает в шок  

  5:  pulse — Пульс  (числовой)  
          - линейный  
          - частота сердечных сокращений в ударах в минуту  
          - отражает состояние сердца: 30–40 — норма для взрослых  
          - редко бывает ниже нормы, хотя у спортивных лошадей может быть 20–25  
          - при болезненных поражениях или шоке пульс обычно повышен  

  9:  mucous membranes — Слизистые оболочки  (категорийный)  
          - субъективная оценка окраски  
          - возможные значения:  
               1 = нормально-розовые  
               2 = ярко-розовые  
               3 = бледно-розовые  
               4 = бледно-цианотичные  
               5 = ярко-красные / воспалённые  
               6 = тёмно-цианотичные  
          - 1 и 2 — нормальное или слегка усиленное кровообращение  
          - 3 — возможно на ранней стадии шока  
          - 4 и 6 — признак серьёзного нарушения циркуляции  
          - 5 — указывает на септицемию  

 11: pain — Боль  (категорийный)  
          - субъективная оценка уровня боли  
          - возможные значения:  
               1 = бодрая, без боли  
               2 = подавленная  
               3 = перемежающаяся лёгкая боль  
               4 = перемежающаяся сильная боль  
               5 = постоянная сильная боль  
          - не следует рассматривать как упорядоченную переменную  
          - чем сильнее боль, тем выше вероятность необходимости операции  
          - обезболивание может частично скрыть уровень боли  

 12: peristalsis — Перистальтика  (категорийный)  
          - показатель активности кишечника; при вздутии или токсичности снижается  
          - возможные значения:  
               1 = гипермоторика  
               2 = нормальная  
               3 = гипомоторика  
               4 = отсутствует  

 13: abdominal distension — Вздутие живота  (категорийный)  
          - ВАЖНЫЙ параметр  
          - возможные значения:  
               1 = нет  
               2 = лёгкое  
               3 = умеренное  
               4 = сильное  
          - сильное вздутие часто требует хирургического вмешательства  

 15: nasogastric reflux — Назогастральный рефлюкс  (категорийный / количественный)  
          - возможные значения:  
               1 = нет  
               2 = > 1 литра  
               3 = < 1 литра  
          - большой объём рефлюкса — признак обструкции кишечника  

 19: packed cell volume — Гематокрит  (числовой)  
          - линейный  
          - процент эритроцитов в крови  
          - норма: 30–50  
          - уровень повышается при обезвоживании или нарушении кровообращения

In [ ]:
import pandas as pd
import numpy as np

attributes_ru = [
    'Операция',
    'Возраст',
    'Номер госпитализации',
    'Ректальная температура',
    'Пульс',
    'Частота дыхания',
    'Температура конечностей',
    'Периферический пульс',
    'Слизистые оболочки',
    'Время капиллярного наполнения',
    'Боль',
    'Перистальтика',
    'Вздутие живота',
    'Назогастральная трубка',
    'Назогастральный рефлюкс',
    'pH назогастрального рефлюкса',
    'Ректальный осмотр — кал',
    'Живот',
    'Гематокрит',
    'Общий белок',
    'Внешний вид жидкости при абдоминоцентезе',
    'Белок при абдоминоцентезе',
    'Исход',
    'Хирургическое поражение',
    'Тип поражения (25)',
    'Тип поражения (26)',
    'Тип поражения (27)',
    'Наличие патологоанатомических данных'
]

horses = pd.read_csv("horse_data.csv")
horses.columns = attributes_ru

horses_important = horses[[
    'Ректальная температура',      # числовой, отражает инфекцию, шок
    'Пульс',                       # числовой, повышается при боли и шоке
    'Слизистые оболочки',          # категориальный, указывает на степень шока и перфузию тканей
    'Боль',                        # категориальный, основной симптом тяжести состояния
    'Перистальтика',               # категориальный, снижение указывает на кишечную непроходимость
    'Вздутие живота',              # категориальный, важнейший показатель, часто требует операции
    'Назогастральный рефлюкс',     # категориальный/количественный, большой объём — признак обструкции
    'Гематокрит'                   # числовой, повышен при обезвоживании и шоке
]]

def separate_print(value, description=''):
    print(value,
          '\n',
          description,
          '\n',
          '---------------------',
          '\n')



# Количественные
horses_rectal_temperature = pd.to_numeric(horses_important['Ректальная температура'], errors='coerce').describe()
separate_print(horses_rectal_temperature,
 """
- линейная  
- в градусах Цельсия  
- повышенная температура возможна при инфекции  
- может снижаться при поздней стадии шока  
- нормальная температура — 37.8  
- этот параметр обычно меняется по мере развития проблемы  
     например: может начинаться нормальной, затем повышаться из-за  
         поражения, снова становиться нормальной, когда лошадь впадает в шок
""")

horses_pulse = pd.to_numeric(horses_important['Пульс'], errors='coerce').describe()
separate_print(horses_pulse,
"""
- линейный  
- частота сердечных сокращений в ударах в минуту  
- отражает состояние сердца: 30–40 — норма для взрослых  
- редко бывает ниже нормы, хотя у спортивных лошадей может быть 20–25  
- при болезненных поражениях или шоке пульс обычно повышен  
""")

horses_packed_cell_volume = pd.to_numeric(horses_important['Гематокрит'], errors='coerce').describe()
separate_print(horses_packed_cell_volume,
"""
- линейный  
- процент эритроцитов в крови  
- норма: 30–50  
- уровень повышается при обезвоживании или нарушении кровообращения
""")

# Категориальные
horses_mucous_membranes = horses_important['Слизистые оболочки'].describe()
separate_print(horses_mucous_membranes,
"""
- субъективная оценка окраски  
- возможные значения:  
     1 = нормально-розовые  
     2 = ярко-розовые  
     3 = бледно-розовые  
     4 = бледно-цианотичные  
     5 = ярко-красные / воспалённые  
     6 = тёмно-цианотичные  
- 1 и 2 — нормальное или слегка усиленное кровообращение  
- 3 — возможно на ранней стадии шока  
- 4 и 6 — признак серьёзного нарушения циркуляции  
- 5 — указывает на септицемию  
""")

horses_pain = horses_important['Боль'].describe()
separate_print(horses_pain,
"""
- субъективная оценка уровня боли  
- возможные значения:  
     1 = бодрая, без боли  
     2 = подавленная  
     3 = перемежающаяся лёгкая боль  
     4 = перемежающаяся сильная боль  
     5 = постоянная сильная боль  
- не следует рассматривать как упорядоченную переменную  
- чем сильнее боль, тем выше вероятность необходимости операции  
- обезболивание может частично скрыть уровень боли  
"""
)

horses_peristalsis = horses_important['Перистальтика'].describe()
separate_print(horses_peristalsis,
"""
- показатель активности кишечника; при вздутии или токсичности снижается  
- возможные значения:  
     1 = гипермоторика  
     2 = нормальная  
     3 = гипомоторика  
     4 = отсутствует  
""")

horses_abdominal_distension = horses_important['Вздутие живота'].describe()
separate_print(horses_abdominal_distension,
"""
- ВАЖНЫЙ параметр  
- возможные значения:  
     1 = нет  
     2 = лёгкое  
     3 = умеренное  
     4 = сильное  
- сильное вздутие часто требует хирургического вмешательства  
""")

horses_nasogastric_reflux = horses_important['Назогастральный рефлюкс'].describe()
separate_print(horses_nasogastric_reflux,
"""
- возможные значения:  
     1 = нет  
     2 = > 1 литра  
     3 = < 1 литра  
- большой объём рефлюкса — признак обструкции кишечника  
""")






# print(base_metrics)

# display(horses_rectal_temperature.head())

count    239.000000
mean      38.166527
std        0.733508
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: Ректальная температура, dtype: float64 
 
- линейная  
- в градусах Цельсия  
- повышенная температура возможна при инфекции  
- может снижаться при поздней стадии шока  
- нормальная температура — 37.8  
- этот параметр обычно меняется по мере развития проблемы  
     например: может начинаться нормальной, затем повышаться из-за  
         поражения, снова становиться нормальной, когда лошадь впадает в шок
 
 --------------------- 

count    275.000000
mean      71.934545
std       28.680522
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: Пульс, dtype: float64 
 
- линейный  
- частота сердечных сокращений в ударах в минуту  
- отражает состояние сердца: 30–40 — норма для взрослых  
- редко бывает ниже нормы, хотя у спортивных лошадей может быть 20–25  
- при болез